In [1]:
from collections import defaultdict
import itertools as it
import numpy as np

from adict import adict

In [2]:
def x(indices):
    """Variable label"""
    i, j = indices
    return f"x{i}_{j}"

def row_regions(N):
    return [[(i, j) for j in range(N)] for i in range(N)]

def column_regions(N):
    return [[(i, j) for i in range(N)] for j in range(N)]

def region_lists(grid: np.array):
    """Lists of region indices from grid"""
    regions = defaultdict(list)
    for (i, j), r in np.ndenumerate(grid):
        regions[r].append((i, j))
    return list(regions.values())

In [3]:
"""Penalties and constraints"""

def penalties(N):
    """Closest neighbours should be different.
    This is a penalty of type x * y, which is 1 iff x = y = 1."""

    Q = adict(int)
    positions = it.product(range(N-1), repeat=2)  # Exclude last row/column

    for (i, j) in positions:
        neighbours = [(i+1, j), (1, j+1), (i+1, j+1)]
        for (k, l) in neighbours:
            Q[x((i,j)), x((k,l))] += 1
        Q[x((i+1, j)), x((i, j+1))]  # Anti-diagonal

    for l in range(N-1):
        Q[x((N-1, l  )), x((N-1, l+1))] += 1
        Q[x((l,   N-1)), x((l+1, N-1))] += 1
        
    return Q

def region_constraint(region: list, stars: int):
    """All points in :region: sum to :stars:."""
    Q = adict(int)
    for (i, j) in region:
        Q[ ( x((i,j)) , x((i,j)) ) ] += (-stars+1)
        for (l, p) in region:
            if (l, p) != (i, j):
                Q[ ( x((i,j)) , x((l,p)) ) ] += 1
    return Q

In [4]:
# Input data

N = 5  # Grid size
nstars = 2  # Number of stars

grid = np.array([[0,1,1,1,1],
                [0,1,1,1,2],
                [0,0,0,2,2],
                [3,0,3,2,2],
                [3,3,3,4,4]])

star = np.array([[0,1,0,0,0],
                [0,0,0,0,1],
                [0,0,1,0,0],
                [1,0,0,0,0],
                [0,0,0,1,0]])

In [5]:
# Build QUBO

Q = penalties(N)

regions = it.chain(
    region_lists(grid),
    row_regions(N),
    column_regions(N),
)

for region in regions:
    Q += region_constraint(region, nstars)

In [6]:
Q

adict(int,
      {('x1_3', 'x0_1'): 1,
       ('x4_2', 'x4_1'): 2,
       ('x1_3', 'x1_3'): -3,
       ('x0_3', 'x1_2'): 1,
       ('x1_0', 'x2_1'): 2,
       ('x3_1', 'x3_1'): -3,
       ('x4_0', 'x4_2'): 2,
       ('x0_3', 'x4_3'): 1,
       ('x1_2', 'x0_3'): 1,
       ('x1_1', 'x2_2'): 1,
       ('x1_2', 'x1_3'): 3,
       ('x3_2', 'x4_3'): 1,
       ('x2_1', 'x2_0'): 2,
       ('x2_4', 'x2_0'): 1,
       ('x2_4', 'x3_4'): 3,
       ('x4_1', 'x4_4'): 1,
       ('x0_0', 'x3_0'): 1,
       ('x0_4', 'x1_2'): 1,
       ('x4_2', 'x4_0'): 2,
       ('x1_2', 'x0_2'): 2,
       ('x1_3', 'x0_2'): 1,
       ('x1_1', 'x2_1'): 2,
       ('x1_1', 'x1_3'): 2,
       ('x1_4', 'x2_4'): 3,
       ('x1_1', 'x0_4'): 1,
       ('x1_0', 'x3_1'): 1,
       ('x0_4', 'x1_3'): 1,
       ('x4_0', 'x3_2'): 1,
       ('x1_2', 'x1_2'): -3,
       ('x3_1', 'x2_2'): 1,
       ('x3_0', 'x4_2'): 1,
       ('x2_3', 'x2_4'): 2,
       ('x0_2', 'x4_2'): 1,
       ('x0_4', 'x3_4'): 1,
       ('x2_1', 'x3_2'): 1,
      